# Pytorch Distributed via Horovod


## Introduction

This recipe shows how to run the [Horovod](https://github.com/uber/horovod) distributed training framework for a [PyTorch](https://pytorch.org/) model using Batch AI.

Currently Batch AI has no native support for Horovod framework, but it's easy to run it using customtoolkit and job preparation command line.


## Details

- Standard Horovod [pytorch_mnist.py](https://raw.githubusercontent.com/uber/horovod/master/examples/pytorch_mnist.py) example will be used;
- pytorch_mnist.py downloads training data on its own during execution;
- The job will be run on the Horovod provded container [uber/horovod:0.13.4-tf1.8.0-torch0.4.0-py3.5](https://hub.docker.com/r/uber/horovod/tags/);
- Horovod and all other dependencies are already installed in the Docker container. See the Tensorflow [Horovod sample](Horovod.ipynb) for guidance on installing at startup. For a production scenario you'd probably want to trim the docker container back to improve job startup performance.
- Standard output of the job will be stored on Azure File Share.

## Instructions

### Install Dependencies and Create Configuration file.
Follow [instructions](/recipes) to install all dependencies and create configuration file.

### Read Configuration and Create Batch AI client

In [35]:
from __future__ import print_function

from datetime import datetime
import sys

from azure.storage.file import FileService
import azure.mgmt.batchai.models as models

# utilities.py contains helper functions used by different notebooks
sys.path.append('../../')
import utilities

cfg = utilities.Configuration('../../configuration.json')
client = utilities.create_batchai_client(cfg)

## 1. Prepare Training Dataset and Script in Azure Storage

### Create File Share

For this example we will create a new File Share with name `batchaisample` under your storage account. This share will be populated with sample scripts and will contain job's output.

**Note** You don't need to create new file share for every cluster. We are doing this in this sample to simplify resource management for you.

In [36]:
azure_file_share_name = 'batchaisample'
service = FileService(cfg.storage_account_name, cfg.storage_account_key)
service.create_share(azure_file_share_name, fail_on_exist=False)
print('Done')

Done


### Deploy Sample Script and Configure the Input Directories

- Download original sample script:

In [37]:
sample_script_url = 'https://raw.githubusercontent.com/uber/horovod/master/examples/pytorch_mnist.py'
utilities.download_file(sample_script_url, 'pytorch_mnist.py')

- Create a folder in the file share and upload the sample script to it.

In [38]:
samples_dir = 'horovod_samples'
service = FileService(cfg.storage_account_name, cfg.storage_account_key)
service.create_directory(
    azure_file_share_name, samples_dir, fail_on_exist=False)
service.create_file_from_path(
    azure_file_share_name, samples_dir, 'pytorch_mnist.py', 'pytorch_mnist.py')

## 2. Create Azure Batch AI Compute Cluster

### Configure Compute Cluster

- For this example we will use a GPU cluster of `STANDARD_NC6` nodes. Number of nodes in the cluster is configured with `nodes_count` variable;
- We will mount file share at folder with name `afs`. Full path of this folder on a computer node will be `$AZ_BATCHAI_MOUNT_ROOT/afs`;
- We will call the cluster `nc6`.
- We will use Low Priority VMs


So, the cluster will have the following parameters:

In [39]:
azure_file_share = 'afs'
nodes_count = 1
cluster_name = 'nc12'

volumes = models.MountVolumes(
    azure_file_shares=[
        models.AzureFileShareReference(
            account_name=cfg.storage_account_name,
            credentials=models.AzureStorageCredentialsInfo(
                account_key=cfg.storage_account_key),
            azure_file_url = 'https://{0}.file.core.windows.net/{1}'.format(
                cfg.storage_account_name, azure_file_share_name),
            relative_mount_path=azure_file_share)
    ]
)

parameters = models.ClusterCreateParameters(
    location=cfg.location,
    vm_size="STANDARD_NC12",
    vm_priority="lowpriority",
    scale_settings=models.ScaleSettings(
        manual=models.ManualScaleSettings(target_node_count=nodes_count)
    ),
    node_setup=models.NodeSetup(
        mount_volumes=volumes
    ),
    user_account_settings=models.UserAccountSettings(
        admin_user_name=cfg.admin,
        admin_user_password=cfg.admin_password,
        admin_user_ssh_public_key=cfg.admin_ssh_key
    )
)

### Create Compute Cluster

In [40]:
_ = client.clusters.create(cfg.resource_group, cluster_name, parameters).result()

### Monitor Cluster Creation

utilities.py contains a helper function allowing to wait for the cluster to become available - all nodes are allocated and finished preparation.

In [43]:
cluster = client.clusters.get(cfg.resource_group, cluster_name)
utilities.print_cluster_status(cluster)

Cluster state: AllocationState.steady Target: 1; Allocated: 1; Idle: 0; Unusable: 0; Running: 0; Preparing: 1; Leaving: 0


## 3. Run Azure Batch AI Training Job

### Configure Input Directories

The job needs to know where to find mnist_replica.py and input MNIST dataset. We will create two input directories for this:
The job needs to know where to find train_mnist.py script (the chainer will download MNIST dataset on its own). So, we will configure an input directory for the script:

In [44]:
input_directories = [
    models.InputDirectory(
        id='SCRIPTS',
        path='$AZ_BATCHAI_MOUNT_ROOT/{0}/{1}'.format(azure_file_share, samples_dir))]

The job will be able to reference those directories using ```$AZ_BATCHAI_INPUT_SCRIPTS``` environment variable.

### Configure Output Directories
We will store standard and error output of the job in File Share:

In [45]:
std_output_path_prefix = '$AZ_BATCHAI_MOUNT_ROOT/{0}'.format(azure_file_share)

The model output will be stored in File Share:

In [46]:
output_directories = [
    models.OutputDirectory(
        id='MODEL',
        path_prefix='$AZ_BATCHAI_MOUNT_ROOT/{0}'.format(azure_file_share),
        path_suffix='Models')]

### Configure Job

- Will use configured previously input and output directories;
- We will use custom toolkit job to run tensorflow_mnist.py on multiple nodes (use node_count parameter to specify number of nodes). Note, Batch AI will create a hostfile for the job, it can be found via ```$AZ_BATCHAI_MPI_HOST_FILE``` environment variable;
- Horovod framework will be installed by job preparation command line;
- Will output standard output and error streams to file share.

You can delete ```container_settings``` from the job definition to run the job directly on host DSVM.

In [47]:
parameters = models.JobCreateParameters(
     location=cfg.location,
     cluster=models.ResourceId(id=cluster.id),
     node_count=1,
     input_directories=input_directories,
     output_directories=output_directories,
     std_out_err_path_prefix=std_output_path_prefix,
     container_settings=models.ContainerSettings(
         image_source_registry=models.ImageSourceRegistry(image='uber/horovod:0.13.4-tf1.8.0-torch0.4.0-py3.5')),
     custom_toolkit_settings = models.CustomToolkitSettings(
         command_line='tail $AZ_BATCHAI_MPI_HOST_FILE; mpirun  -mca btl_tcp_if_exclude docker0,lo --allow-run-as-root --tag-output --hostfile $AZ_BATCHAI_MPI_HOST_FILE python $AZ_BATCHAI_INPUT_SCRIPTS/pytorch_mnist.py'))

### Create a training Job and wait for Job completion


In [48]:
job_name = datetime.utcnow().strftime('horovod_pytorch_%m_%d_%Y_%H%M%S')
job = client.jobs.create(cfg.resource_group, job_name, parameters).result()
print('Created Job: {}'.format(job.name))

Created Job: horovod_pytorch_06_12_2018_024934


### Wait for Job to Finish
The job will start running when the cluster will have enough idle nodes. The following code waits for job to start running printing the cluster state. During job run, the code prints current content of stderr.txt.

**Note** Execution may take several minutes to complete.

In [49]:
utilities.wait_for_job_completion(client, cfg.resource_group, job_name, cluster_name, 'stdouterr', 'stderr.txt')

Cluster state: AllocationState.steady Target: 1; Allocated: 1; Idle: 0; Unusable: 0; Running: 1; Preparing: 0; Leaving: 0
Job state: running ExitCode: None
Waiting for job output to become available...
[1,1]<stddiag>:[28247c31031f45d7acb4b393b8082300000000:00088] Read -1, expected 10000, errno = 1
[1,1]<stddiag>:[28247c31031f45d7acb4b393b8082300000000:00088] Read -1, expected 32000, errno = 1
[1,0]<stderr>:/mnt/batch/tasks/shared/LS_root/mounts/afs/horovod_samples/pytorch_mnist.py:80: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
[1,0]<stderr>:  return F.log_softmax(x)
[1,0]<stderr>:/mnt/batch/tasks/shared/LS_root/mounts/afs/horovod_samples/pytorch_mnist.py:116: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number
[1,0]<stderr>:  100. * batch_idx / len(train_loader), loss.data[0]))
[1,1]<stderr>:/mnt/batch/tasks/sha

### Download stdout.txt and stderr.txt files for the Job and job preparation command

In [ ]:
files = client.jobs.list_output_files(cfg.resource_group, job_name,
                                      models.JobsListOutputFilesOptions(outputdirectoryid='stdouterr')) 
for f in list(files):
    if f.download_url:
        utilities.download_file(f.download_url, f.name)
print('All files downloaded')

## 4. Clean Up (Optional)

### Delete the Job

In [32]:
_ = client.jobs.delete(cfg.resource_group, job_name)

### Delete the Cluster
When you are finished with the sample and don't want to submit any more jobs you can delete the cluster using the following code.

In [33]:
_= client.clusters.delete(cfg.resource_group, cluster_name)

### Delete File Share
When you are finished with the sample and don't want to submit any more jobs you can delete the file share completely with all files using the following code.

In [34]:
service = FileService(cfg.storage_account_name, cfg.storage_account_key)
service.delete_share(azure_file_share_name)

True